# 3절. Selenium을 이용한 웹 데이터 수집
- 구글에서 'selenium python'검색
https://selenium-python.readthedocs.io/getting-started.html
- pip install selenium

- **chrome버전을 확인하고 맞는 chrome driver버전을 다운로드** 

In [1]:
from selenium import webdriver
driver= webdriver.Chrome("D:/bigdata(psa)/selenium/chromedriver.exe")    # /하나 혹은 \\ (역슬래시 두 개)
#chromedriver.exe가 쥬피터노트북파일 여기서  bigdata(psa) src  8_python안에 있으면  Chrome() 안에 아무것도 안써도됨 

In [2]:
driver.get('http://python.org')

In [5]:
elem=driver.find_element_by_name('q')   #find에  shift+tab 키   
elem.clear()   # 검색창안에 입력한 내용을 지우고 
elem.send_keys("pycon")    #그러면 브라우저 검색창에서 pycon이 나타남 

In [6]:
#pycon를 쳤으면 enter를 눌러라
from selenium.webdriver.common.keys import Keys
elem.send_keys(Keys.RETURN) # Enter키 

In [9]:
result_list=driver.find_elements_by_css_selector("form h3>a")   #결과 여러개 있어서 find_element가 아니고  find_elements  
for result in result_list:
    print("{%s-%s"%(result.text,result.get_attribute("href")))

{PSF PyCon Trademark Usage Policy-https://www.python.org/psf/trademarks/pycon
{Conferences and Workshops-https://www.python.org/community/workshops
{PyCon Italia 2016 (PyCon Sette)-https://www.python.org/events/python-events/378/
{2008-04-14 PSF Board Meeting Minutes-https://www.python.org/psf/records/board/minutes/2008-04-14
{2012-07-16 PSF Board Meeting Minutes-https://www.python.org/psf/records/board/minutes/2012-07-16
{2013-02-06 PSF Board Meeting Minutes-https://www.python.org/psf/records/board/minutes/2013-02-06
{PyCon Home at python.org-https://www.python.org/community/pycon
{PyCon Australia 2013-https://www.python.org/events/python-events/57/
{PyCon AU 2019-https://www.python.org/events/python-events/776/
{PyCon Australia 2014-https://www.python.org/events/python-events/10/
{PyCon Ireland 2012-https://www.python.org/events/python-events/76/
{PyCon Ireland 2016-https://www.python.org/events/python-events/429/
{PyCon AU 2018-https://www.python.org/events/python-events/696/
{PyCon

In [10]:
#브라우저 종료
driver.close()

## **맞춤법 검사기(네이버 맞춤법 검사기 이용)**

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
import time 

driver = webdriver.Chrome("D:/bigdata(psa)/selenium/chromedriver.exe")
driver.get("http://www.naver.com")

In [10]:
elem = driver.find_element_by_name("query")  # name="query"가 있는 곳에 "맞춤법 검사기" 치고, 그 페이지를 호출
elem.send_keys("맞춤법 검사기")
elem.send_keys(Keys.RETURN)

- 네이버 맞춤법 검사기에 "한녕하세요.방갑습니다."라는 text가 들어감

In [11]:
textarea= driver.find_element_by_class_name("txt_gray")  #find치고tab치면  find_element_by_class_name  나옴
#검색창 소스를 보면 class ="txt_gray"
textarea.send_keys("한녕하세요. 방갑습니다.")

- '검사하기' 버튼이 클릭됨

In [13]:
button = driver.find_element_by_class_name("btn_check")  # 검사하기 버튼 소스
button.click()

In [17]:
soup = BeautifulSoup(driver.page_source,"html.parser")
result = soup.select_one("p._result_text.stand_txt").text   # .text를해야 내용이 나옴 
result

'맞춤법 검사를 원하는 단어나 문장을 입력해 주세요. 한 멍하세요. 반갑습니다.'

- **<최종 버전>**

In [23]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
import time 

driver = webdriver.Chrome("D:/bigdata(psa)/selenium/chromedriver.exe")
driver.get("http://www.naver.com")

elem = driver.find_element_by_name("query")  # name="query"가 있는 곳에 "맞춤법 검사기" 치고, 그 페이지를 호출
elem.send_keys("맞춤법 검사기")
elem.send_keys(Keys.RETURN)
time.sleep(1)     #잠깐 있어라. 그래야 다음을 넘어감  ***** 추가된 내용 **********************

textarea= driver.find_element_by_class_name("txt_gray")  #find치고tab치면  find_element_by_class_name  나옴
#검색창 소스를 보면 class ="txt_gray"
textarea.send_keys(Keys.CONTROL,"a ")                    #***** 추가된 내용 **********************
textarea.send_keys("한녕하세요. 방갑습니다.")

button = driver.find_element_by_class_name("btn_check")  # 검사하기 버튼 소스
button.click()
time.sleep(2)  #검사하기 클릭 후 잠시 후에 결과 크롤링    #***** 추가된 내용 **********************
soup = BeautifulSoup(driver.page_source,"html.parser")
result = soup.select_one("p._result_text.stand_txt").text   # .text를해야 내용이 나옴 
print(result)


안녕하세요. 반갑습니다.


## **맞춤법감사가 안 된 text파일을 맞춤법 검사가 완료된 text파일로 output** 

In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
import time 

with open("data/ch14.txt","r",encoding="utf-8") as f:
    text = f.read()
print(len(text))
ready_list=[]  #text를 '500글자가 있는 바로 앞 sapce 잘라서' (글자중간에 자르면 맞춤법 수정이 안될 수 있기때문에) list에 넣음 
while len(text)>500:
    temp_str = text[:500]
    last_space_idx = temp_str.rfind(' ')            #rfind는 right find 오른쪽부터 찾는다   '. space'해야 한문장이 끝났다
    ready_list.append(text[:last_space_idx])  #500자
    text = text[last_space_idx:] #readY_list에 append한 앞부분은 날림
ready_list.append(text)  #마지막 500문자 안된 나머지의 문자들을 append 
print(len(ready_list))
print((ready_list))

#webdriver, chrome, ready_list에 있는 아이를 넣는다. for문 
#결과. 하나의 text file
#file로 output 
driver = webdriver.Chrome("D:/bigdata(psa)/selenium/chromedriver.exe")
driver.get("http://www.naver.com")

elem = driver.find_element_by_name("query")  # name="query"가 있는 곳에 "맞춤법 검사기" 치고, 그 페이지를 호출
elem.send_keys("맞춤법 검사기")
elem.send_keys(Keys.RETURN)
time.sleep(1)     #잠깐 있어라. 그래야 다음을 넘어감  ***** 추가된 내용 **********************

textarea= driver.find_element_by_class_name("txt_gray")  #find치고tab치면  find_element_by_class_name  나옴
#검색창 소스를 보면 class ="txt_gray"
results =""
for ready in ready_list:
    textarea.send_keys(Keys.CONTROL,"a")  #ctrl a는 기존에 있는것을 지운다는 뜻
    textarea.send_keys(ready)
    button = driver.find_element_by_class_name("btn_check")  # 검사하기 버튼 소스
    button.click()
    time.sleep(2)  #검사하기 클릭 후 잠시 후에 결과 크롤링    #***** 추가된 내용 **********************
    soup = BeautifulSoup(driver.page_source,"html.parser")
    result = soup.select_one("p._result_text.stand_txt").text   # .text를해야 내용이 나옴 
    results += result.replace('.','.\n')
#print(results)
with open('data/ch14_checked.txt','w',encoding='utf-8') as f:
    f.write(results)
driver.close()   #알아서 browser 닫힘 

887
2
['한녕하세요. \n\n블룸버그을 비롯한 외신들은 이반 조치에 대해 “미국 제재로 어려움을 겪고 있는 화웨이가 새로운 수익원을 만들려는 것이다"고 분석했다.\n\n실제로 화웨이는 미국 제재 이후 스마트폰 시장에서 큰 어려움을 겪고 있다. 한 때 1위 자리까지 넘보던 화웨이는 3위권 밖으로 밀려났다.\n\n■ 퀄컴이나 에릭슨·노키아보다 훨씬 좋은 조건\n\n허지만 특허전문 사이트 포스페이턴츠는 화웨이의 특허료 부과 조건이 상당이 합리적인 편이라고 평가했다.\n\n이 사이트를 운영하고 있는 플로리언 뮐러는 "대당 최대 2.5달러를 부과하겠다는 화웨이의 조건을 보고 안도했다”면서 “정치적인 지형과 관계 없이 화웨이는 여전히 제푸메 초점을 맞춘 혁신가 그룹에 속해 있다는 사실을 확인했다”고 평가했다.\n\n그는 특히 “에릭슨, 노키아 등의 지적재산권 정책과 달리 화웨이는 자신들의 특허권이 스마트폰, 커넥티드카, 그리고 좀더넓게는 사물인터넷 분야의 혁신에 도움되는 방양으로 라이선스되길 원하는 것 같다"고', ' 덧붙였다.\n\n포스페이턴츠는 5G 단말기 한 대당 2.50달러라는 화웨이의 로열티는 다른 업체들에 비해서도 낮은 편이라고 지적했다.\n\n미국 연방거래위원회(FTC)와 소송 당시 공개된 자료에 따르면 퀄컴은 5G 단말기 한대당 13달러까지 요구했다고 포스페이턴츠가 전했다.\n\n또 노키아는 대당 로열티 상한선은 3유로(3.59달러)다. 에릭슨은 구체적인 기준이 공개되지 않았지만 노키아에 비해선 월등히 높은 편인 것으로 알려졌다.\n\n포스페이턴츠는 이 같은 사실을 지적하면서 “대당 2.50달러라는 화웨이의 5G 로열티 상한선은 올해 자주 거론될 것 같다”고 전망했다. 특히 에릭슨과 소송 중인 삼성이 법정에서 화웨이의 5G 필수표준특허 라이선스 조건을 적극 활용할 것으로 예상했다.\n\n감사함니다. 방갑습니다. ']


번역

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
import time 

with open("data/ch14.txt","r",encoding="utf-8") as f:
    text = f.read()
print(len(text))
ready_list=[]  #text를 '500글자가 있는 바로 앞 sapce 잘라서' (글자중간에 자르면 맞춤법 수정이 안될 수 있기때문에) list에 넣음 
while len(text)>500:
    temp_str = text[:500]
    last_space_idx = temp_str.rfind('. ')            #rfind는 right find 오른쪽부터 찾는다   '. space'해야 한문장이 끝났다
    ready_list.append(text[:last_space_idx+1])  #500자                            # idx에 +1를 해야  .이 안빠짐
    text = text[last_space_idx+1:] #readY_list에 append한 앞부분은 날림
ready_list.append(text)  #마지막 500문자 안된 나머지의 문자들을 append 
print(len(ready_list))
print((ready_list))

#webdriver, chrome, ready_list에 있는 아이를 넣는다. for문 
#결과. 하나의 text file
#file로 output 
driver = webdriver.Chrome("D:/bigdata(psa)/selenium/chromedriver.exe")
driver.get("http://www.naver.com")

elem = driver.find_element_by_name("query")  # name="query"가 있는 곳에 "맞춤법 검사기" 치고, 그 페이지를 호출
elem.send_keys("맞춤법 검사기")
elem.send_keys(Keys.RETURN)
time.sleep(1)     #잠깐 있어라. 그래야 다음을 넘어감  ***** 추가된 내용 **********************

textarea= driver.find_element_by_class_name("txt_gray")  #find치고tab치면  find_element_by_class_name  나옴
#검색창 소스를 보면 class ="txt_gray"
results =""
for ready in ready_list:
    textarea.send_keys(Keys.CONTROL,"a")  #ctrl a는 기존에 있는것을 지운다는 뜻
    textarea.send_keys(ready)
    button = driver.find_element_by_class_name("btn_check")  # 검사하기 버튼 소스
    button.click()
    time.sleep(2)  #검사하기 클릭 후 잠시 후에 결과 크롤링    #***** 추가된 내용 **********************
    soup = BeautifulSoup(driver.page_source,"html.parser")
    result = soup.select_one("p._result_text.stand_txt").text   # .text를해야 내용이 나옴 
    results += result.replace('.','.\n')
#print(results)
with open('data/ch14_checked.txt','w',encoding='utf-8') as f:
    f.write(results)
driver.close()   #알아서 browser 닫힘 